# NeOS-A Log 분석
- neos_db query 직접추출한 자료로 작업

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame as df
from pandas import json_normalize
from matplotlib import font_manager, rc
import os
import glob  
import sys
from tqdm.notebook import tqdm
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d')

# neos_db 추가 추출된 자료 업로드

In [2]:
df_start = pd.read_csv('./neos_nugu_db/nugu_20210307.txt', delimiter = '\t',names=['date','UserID','LogType','SceneId','Log'])
log_data = df_start['Log'].reset_index(drop=True)

# ★ json 구조를 데이터 프레임으로 저장
- 엄청 오래걸리는 작업 : 주의(★★★★★)

In [3]:
df_start = df_start[df_start['Log'].notnull()]

In [4]:
df1 = df()
for i in tqdm(range(len(log_data))):
    log_data[i] = log_data[i].replace('true','"true"')
    log_data[i] = log_data[i].replace('false','"false"')
    df1=df1.append(json_normalize(eval(log_data[i])), ignore_index=True)

In [5]:
df_start.reset_index(drop=True, inplace = True)
df3 = pd.concat([df_start,df1], axis=1)

# 기존 추출 작업한 엑셀 데이터 업로드

In [6]:
# df_past = pd.read_excel('./neos_nugu_db/neos_nugu_db_excel/neos_nugu_가공자료_20201221.xlsx', encoding = 'euc-kr')

# df_past.drop(columns=['Unnamed: 0'], inplace = True)

# df4 = pd.concat([df_past,df3]).reset_index().drop(columns = ['index'] )

# df4.head()

# 중간 엑셀저장

In [7]:
# df4.to_excel('./neos_nugu_db/neos_nugu_db_excel/neos_nugu_가공자료_'+now+'.xlsx', encoding='euc-kr')

In [8]:
df3.to_excel('./neos_nugu_db/neos_nugu_db_excel/neos_nugu_가공자료_'+now+'.xlsx', encoding='euc-kr')

# Action name 기반으로 Funciton 생성

In [9]:
# df5 = df4.fillna('')
df5 = df3.fillna('')

In [10]:
##필드값 합치기 
df6 = df5.filter(regex='type$', axis = 1)
df5['RequestType'] = df6.apply('/'.join, axis=1)

In [11]:
## 합친 필드값에서 '/' 쓸데없는 값 없애기
df5['RequestType'] = df5['RequestType'].str.replace(pat=r'[/]', repl= r' ', regex=True)
df5['RequestType'] = df5['RequestType'].str.replace(pat=r'\s+', repl= r' ', regex=True) 

df5['RequestType']=df5['RequestType'].str.strip()


In [17]:
conditionlist = [
    (df5['action.actionName'] == "01open_door"),
    (df5['action.actionName'] == '02register_work'),
    (df5['action.actionName'] == '03ask_visitor'),
    (df5['action.actionName'] == '04ask_accessinfo'),    
    (df5['action.actionName'] == '05menu_workhistory'),
    (df5['action.actionName'] == '06navigation'),
    (df5['action.actionName'] == '07change_station'),
    (df5['action.actionName'] == '08go'),
    (df5['action.actionName'] == '09common_nogo_popupclose'),
    (df5['action.actionName'] == '11common_searchresult_number'),
    (df5['action.actionName'] == '13ask_guide'),
    (df5['action.actionName'] == '12ask_who') & ((df5['RequestType']).str.contains ('WORKER')),
    (df5['action.actionName'] == '12ask_who') & ((df5['RequestType']).str.contains ('VACATIONER')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('WORKLIST')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('MYWORK')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('NOTICE')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('SCHEDULER')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('TROUBLE')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('WORKHISTORY')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('ACCESSHISTORY')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('ALARMHISTORY')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('HISTORYINFO')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('WORKLOG')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('VISITOR')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('ACCESSINFO')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('NEARSTATION')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('NEARWORKLIST')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('NEARRU')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_COUNTY')& ((df5['RequestType']).str.contains ('REPEATER'))),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_COUNTY')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_CITY')& ((df5['RequestType']).str.contains ('REPEATER'))),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_CITY')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('WORKTIME')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('WORKER')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('VACATIONER')),
    (df5['action.actionName'] == '10askinfo') & ((df5['RequestType']).str.contains ('GUIDE')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('WORKLIST')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('MYWORK')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('NOTICE')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('SCHEDULER')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('TROUBLE')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('WORKHISTORY')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('ACCESSHISTORY')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('ALARMHISTORY')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('HISTORYINFO')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('WORKLOG')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('VISITOR')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('ACCESSINFO')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('NEARSTATION')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('NEARWORKLIST')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('NEARRU')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_COUNTY')& ((df5['RequestType']).str.contains ('REPEATER'))),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_COUNTY')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_CITY')& ((df5['RequestType']).str.contains ('REPEATER'))),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('BID_LOC_CITY')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('WORKTIME')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('WORKER')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('VACATIONER')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('GUIDE')),
    (df5['action.actionName'] == '14small_event'),
    (df5['action.actionName'] == '16ask_stationinfo'),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('STATIONINFO')),
    (df5['action.actionName'] == '10common_askinfo') & ((df5['RequestType']).str.contains ('LIVEALARM'))
]
choicelist = ['출입문제어', '작업등록', '출입이력 조회', '출입정보 조회', '업무일지 작성', '국소 길안내', '출동국소 변경', '출동/선택', '확인/닫기','국소검색결과 선택', '도움말', 
              '근무자 조회', '휴가자 조회','오늘의업무 조회', '계획업무 조회', '알리미 조회', '스케쥴러 조회', 'OOS 조회', '작업이력 조회', '출입이력 조회', '알람이력 조회', 
              '이력정보 조회', '작업일지 조회', '출입이력 조회', '출입정보 조회', '주변기지국 조회', '주변업무 조회', '주변중계기 조회', '중계기검색', '기지국검색','중계기검색(미지원)', '기지국검색(미지원)', '근무시간 조회', '근무자 조회', '휴가자 조회', '도움말',
              '오늘의업무 조회', '계획업무 조회', '알리미 조회', '스케쥴러 조회', 'OOS 조회', '작업이력 조회', '출입이력 조회', '알람이력 조회', 
              '이력정보 조회', '작업일지 조회', '출입이력 조회', '출입정보 조회', '주변기지국 조회', '주변업무 조회', '주변중계기 조회', '중계기검색', '기지국검색','중계기검색(미지원)', '기지국검색(미지원)', '근무시간 조회', '근무자 조회', '휴가자 조회', '도움말','이벤트',
             '환경정보','환경정보','실시간 알람']


In [18]:
df5['Function'] = np.select(conditionlist, choicelist, default='Unknown')

In [21]:
df_fin=df5[['date','UserID','SceneId','action.actionName',
            'RequestType','Function',#'event.text',
            'context.supportedInterfaces.Extension.data.SetData.EquipCode',
     'context.supportedInterfaces.Extension.data.SetData.LocalName','context.supportedInterfaces.Extension.data.SetData.MyLatitd',
    'context.supportedInterfaces.Extension.data.SetData.MyLongitd','context.supportedInterfaces.Extension.data.SetStationId']]

# 1차 가공자료 엑셀 백업

In [22]:
df_fin.to_excel('./neos_nugu_db/neos_nugu_db_excel/1차 가공/neos_nugu_1차_가종자료'+now+'.xlsx', encoding='euc-kr')

# 보고서 자료 추출

# excel merge한거 다시 불러오기 

In [23]:
df_fin = pd.read_excel('./neos_nugu_db/neos_nugu_db_excel/1차 가공/neos_nugu_1차_가종자료_merge.xlsx', encoding= 'euc-kr')

In [24]:
df_fin.drop(columns=['Unnamed: 0'], inplace = True)

In [48]:
user_info = pd.read_excel('./neos_nugu_db/neos_대상구분.xlsx', encoding = 'euc-kr')
user_info_2021 = pd.read_excel('./neos_nugu_db/neos_대상구분_202101.xlsx', encoding = 'euc-kr')

weekly_df = pd.read_excel('./휴일_주차구분.xlsx', encoding = 'euc-kr')
df_opti = df_fin[['date', 'UserID', 'SceneId', 'action.actionName', 'RequestType', 'Function','event.text']]
df_opti['date_day'] = df_opti['date'].astype('str').apply(lambda x : x[:10])

In [49]:
df_opti['date_year'] = df_opti['date'].astype('str').apply(lambda x : x[:4])

In [50]:
df_opti.head()

,date,UserID,SceneId,action.actionName,RequestType,Function,event.text,date_day,date_year
0,2020-10-04 10:48:14.133,N1101208,DEFAULT,10askinfo,WORKTIME app.neos.android,근무시간 조회,NaN,2020-10-04,2020
1,2020-10-05 07:27:40.333,N1101208,DEFAULT,10askinfo,app.neos.android TROUBLE,OOS 조회,NaN,2020-10-05,2020
2,2020-10-05 08:55:38.160,N1103225,DEFAULT,10askinfo,app.neos.android BID_LOC_COUNTY,기지국검색,NaN,2020-10-05,2020
3,2020-10-05 08:55:48.493,N1103225,DEFAULT,10askinfo,app.neos.android BID_LOC_COUNTY,기지국검색,NaN,2020-10-05,2020
4,2020-10-05 08:56:24.027,N1103225,DEFAULT,10askinfo,app.neos.android,Unknown,NaN,2020-10-05,2020


In [51]:
df_opti_2020 = df_opti[df_opti['date_year']=='2020']

In [52]:
df_opti_2021 = df_opti[df_opti['date_year']=='2021']

In [53]:
df_opti_2021.shape

(37927, 9)

In [54]:
df_opti_2020.shape

(124168, 9)

In [55]:
df_work1 = pd.merge(df_opti_2020,user_info, left_on='UserID', right_on='사번', how = 'left')
df_work1_1 = pd.merge(df_opti_2021,user_info_2021, left_on='UserID', right_on='사번', how = 'left')

In [56]:
df_work1_2 = pd.concat([df_work1,df_work1_1])

In [57]:
df_work1_2.shape

(162095, 18)

In [58]:
# df_work1 = pd.merge(df_opti,user_info, left_on='UserID', right_on='사번', how = 'left')



df_work_3 = df_work1_2[df_work1_2['제외대상']=='구성원']
df_work_3 = df_work_3[df_work_3['미대상팀']=='대상']
df_work2 = df_work_3.drop(columns=['사번','직급','연락처','제외대상','미대상팀'])

# 분단위 분리, day 분리, 시간 분리
# df_work2['date_day'] = df_work2['date'].astype('str').apply(lambda x : x[:10])

In [59]:
df_work2.head()

,date,UserID,SceneId,action.actionName,RequestType,Function,event.text,date_day,date_year,본부,팀,직위,이름
0,2020-10-04 10:48:14.133,N1101208,DEFAULT,10askinfo,WORKTIME app.neos.android,근무시간 조회,NaN,2020-10-04,2020,인천본부,일산품질개선팀,파트장,김민규
1,2020-10-05 07:27:40.333,N1101208,DEFAULT,10askinfo,app.neos.android TROUBLE,OOS 조회,NaN,2020-10-05,2020,인천본부,일산품질개선팀,파트장,김민규
2,2020-10-05 08:55:38.160,N1103225,DEFAULT,10askinfo,app.neos.android BID_LOC_COUNTY,기지국검색,NaN,2020-10-05,2020,전북본부,전주품질개선팀,Manager,채신기
3,2020-10-05 08:55:48.493,N1103225,DEFAULT,10askinfo,app.neos.android BID_LOC_COUNTY,기지국검색,NaN,2020-10-05,2020,전북본부,전주품질개선팀,Manager,채신기
4,2020-10-05 08:56:24.027,N1103225,DEFAULT,10askinfo,app.neos.android,Unknown,NaN,2020-10-05,2020,전북본부,전주품질개선팀,Manager,채신기


In [60]:
df_work2.tail()

,date,UserID,SceneId,action.actionName,RequestType,Function,event.text,date_day,date_year,본부,팀,직위,이름
37921,2021-03-07 17:59:44.603,N1104215,DEFAULT,10askinfo,app.neos.android TROUBLE,OOS 조회,2018Z4828,2021-03-07,2021,강남본부,강동품질개선팀,Manager,이재훈
37922,2021-03-07 18:03:11.933,N1104215,DEFAULT,17comebackhome,app.neos.android,Unknown,2018Z4828,2021-03-07,2021,강남본부,강동품질개선팀,Manager,이재훈
37923,2021-03-07 18:40:55.220,N1101347,DEFAULT,01open_door,app.neos.android,출입문제어,200622348,2021-03-07,2021,전남본부,순천품질개선팀,Manager,김정욱
37925,2021-03-07 21:03:53.013,N1101120,DEFAULT,06navigation,app.neos.android,국소 길안내,2006Z4099,2021-03-07,2021,전북본부,전주품질개선팀,Manager,권영길
37926,2021-03-07 22:40:30.950,N1102103,DEFAULT,17comebackhome,app.neos.android,Unknown,2014Y1459,2021-03-07,2021,경남본부,김해품질개선팀,Manager,장윤호


# 활용율
분석할때 user_info 기준 활용율 활용

In [61]:
user_info2 = user_info_2021[(user_info_2021['제외대상'] == '구성원') & (user_info_2021['미대상팀']=='대상')]
user_info3 = user_info2.groupby(['본부']).size().reset_index(name = '인원수')

In [62]:
print('총구성원 : ',user_info[(user_info['제외대상'] == '구성원') ].shape[0])
print('서비스 대상 : ',user_info2.shape[0])
print('누적사용자 수 : ',df_work2.drop_duplicates(['UserID'], keep='last').shape[0])

총구성원 :  2329
서비스 대상 :  1497
누적사용자 수 :  1340


In [63]:
df_work3 = pd.merge(df_work2,weekly_df, left_on='date_day', right_on='date', how='left')

In [72]:
df_work3['사용구분'] = "음성"

In [74]:
df_navi_start = pd.read_csv('./neos_nugu_db/neos_navigation_db/nugu_navigation_20210307.csv',names=['Seq','UserID','DeviceType',
                                                                                               'DeviceId','LogType','SceneId','Log',
                                                                                              'RegtDate'])
df_navi_opti = df_navi_start[['UserID', 'SceneId','RegtDate']]
df_navi_opti['date_day'] = df_navi_opti['RegtDate'].astype('str').apply(lambda x : x[:10])

conditionlist = [
    (df_navi_opti['SceneId']=='TMAP'),
    (df_navi_opti['SceneId']=='ACCESSINFO'),
    (df_navi_opti['SceneId']=='STATIONINFO'),
    (df_navi_opti['SceneId']=='MENU_WORKLOG'),
    (df_navi_opti['SceneId']=='HISTORY_ALARM'),
    (df_navi_opti['SceneId']=='REGISTERWORK_CONFIRM'),
    (df_navi_opti['SceneId']=='LIVEALARM'),
    (df_navi_opti['SceneId']=='DOOR_OPEN'),
]

choicelist = ['국소 길안내','출입정보 조회', '환경정보', '업무일지 작성','이력정보 조회','작업등록','실시간 알람','출입문제어']
df_navi_opti['Function'] = np.select(conditionlist,choicelist, default = 'Unknown')

In [76]:
df_navi_work1 = pd.merge(df_navi_opti,user_info_2021, left_on='UserID', right_on='사번', how = 'left')
df_navi_work2 = pd.merge(df_navi_work1,weekly_df, left_on='date_day', right_on='date', how='left')

In [78]:
df_navi_work2.reset_index(drop=True,inplace=True)
df_navi_work3 = df_navi_work2[df_navi_work2['제외대상']=='구성원']
df_navi_work3 = df_navi_work3[df_navi_work3['미대상팀']=='대상']
df_navi_work3 = df_navi_work3.drop(columns=['사번','직급','연락처','제외대상','미대상팀','date','SceneId'])

In [79]:
df_navi_work3.rename(columns={'RegtDate':'date_x'},inplace=True)

In [82]:
df_navi_work3['사용구분'] = "Navi"

In [85]:
df_work3.drop(columns=['SceneId','RequestType','date_y'], inplace=True)

In [87]:
df_work4 = pd.concat([df_work3,df_navi_work3])

In [88]:
df_work4.to_excel('./neos_a_활용(사이드바포함)_'+now+'.xlsx', encoding='euc-kr')

# 주차별 본부활용

In [89]:
df_deduplication = df_work4.drop_duplicates(['주차','본부','UserID'], keep='first')
df_area_weekly = df_deduplication.groupby(['주차','본부']).size().reset_index(name='count')

In [90]:
import xlsxwriter
import openpyxl
from openpyxl.workbook import Workbook
from openpyxl.styles import Font, Fill

import warnings
warnings.filterwarnings('ignore')

In [91]:
df_area_weekly.to_excel('./주차별 본부사용자_'+now+'.xlsx', sheet_name='주차별 본부사용자수', encoding='euc-kr')

In [94]:
df_work4.groupby(['date_day']).size().reset_index(name='건수').to_excel('./일별발화건수_'+now+'.xlsx', sheet_name='date_count', encoding='euc-kr')

In [95]:
df_work4.groupby('주차').size().reset_index(name='건수')

,주차,건수
0,10월 1주차,2824
1,10월 2주차,4988
2,10월 3주차,5852
3,10월 4주차,7754
4,10월 휴일,1030
5,11월 1주차,10038
6,11월 2주차,21679
7,11월 3주차,18332
8,11월 4주차,13467
9,11월 휴일,2859


# 다시 해보자 ㅠㅠ

In [ ]:
filename = './NeOS-A_Report2020-12-04.xlsx'
book = openpyxl.load_workbook(filename)

In [ ]:
sheet_area_weekly = book.create_sheet('주차별 본부사용자')
book.save(filename)

In [ ]:
writer = pd.ExcelWriter('./NeOS-A_Report2020-12-04.xlsx', sheet_name='date_count', engine='xlsxwriter')
df_area_weekly.to_excel(writer, sheet_name='주차별 본부사용자', startrow=1)